# BPR Bayesian Personalized Ranking

---

ThetaLog.com

In [1]:
# Load các thư viện cần thiết
import os
import math
import pathlib
import numpy as np
import pandas as pd
from time import time
from urllib.request import urlopen
from zipfile import ZipFile
from scipy.sparse import csr_matrix, dok_matrix
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from datetime import datetime
import torch
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

np.random.seed(12)

In [2]:
train_path = './MINDsmall_train/behaviors.tsv'

output_train_path = './datafile/small_train.csv'

result_train_file = 'small_train'

user_min = 30
item_min = 30

test_ratio = 0.9

alpha=0.001
lamb=0.02
k=60
n_iters=300000
initial_value = 0.5

In [3]:
class Similarity:
    def __init__(self, type='train'):
        self.path = f'./MINDsmall_{type}/news.tsv'
        self.df = pd.read_csv(self.path, sep='\t', names=['News_ID', "Category", "SubCategory", "Title", "Abstract", "URL", "CreatedAt", "Body"], skiprows = 1)
        self.df['title_body'] = self.df[['Title', 'Body']].apply(
            lambda x: ' '.join(x.dropna().astype(str)),
            axis=1
        )
        self.df.drop(columns=["Category", "SubCategory", "Title", "Abstract", "URL", 'CreatedAt'])
        self.calc_tf_idf()
        self.create_news_index()

    def create_news_index(self):
        self.mark = dict()
        for id, item in self.df['News_ID'].iteritems():
            self.mark[int(item[1:])] = id

    def calc_tf_idf(self):
        vectorizer = TfidfVectorizer(stop_words='english')
        corpus = self.df['title_body'].dropna()
        self.tf_idf_matrix = vectorizer.fit_transform(corpus)

    def similarity(self, item_i, item_j):
        x = self.mark[item_i]
        y = self.mark[item_j]
        return cosine_similarity(self.tf_idf_matrix[x], self.tf_idf_matrix[y])

In [4]:
def load_behaviors(path):
    df = pd.read_csv(
        path,
        sep = "\t",
        names = ["id", "user_id", "time", "history", "impressions"]
    )
    return df

def write_file(df, path):
    t_start = time()
    print("Processing", path)
    with open(path, "w") as file:
        for i, row in df.iterrows():
            for new_id in str(row.history).split(' '):
                if row.user_id[1:].isnumeric() and new_id[1:].isnumeric():
                    file.write(row.user_id[1:] + "," + new_id[1:] + "\n")

    t_end = time()
    print("Processed in: {:.2f} Seconds".format(t_end - t_start))

def write_data_file(train_path, output_train_path):
    train_df = load_behaviors(train_path)

    write_file(train_df, output_train_path)

In [5]:
# write_data_file(train_path, output_train_path)

In [6]:
def load_file(path):
    df = pd.read_csv(
        path,
        header=None,
        names=['user_id', 'item_id']
    )
    df.reindex(columns = ['user_id', 'item_id'])
    return df.reset_index(drop = True)

def load_data(train_path):
    train_df = load_file(train_path)
    
    return train_df

In [7]:
train_df = load_data(output_train_path)

In [8]:
def filter_df(df, user_min, item_min):
    if df is None:
        return

    t_start = time()

    user_counts = df.groupby("user_id").size()
    user_subset = np.in1d(
        df.user_id, user_counts[user_counts >= item_min].index
    )

    filter_df = df[user_subset].reset_index(drop=True)

    # find items with 10 or more users
    item_counts = filter_df.groupby("item_id").size()
    item_subset = np.in1d(
        filter_df.item_id, item_counts[item_counts >= user_min].index
    )

    filter_df = filter_df[item_subset].reset_index(drop=True)

    user_counts = filter_df.groupby("user_id").size()
    user_subset = np.in1d(filter_df.user_id, user_counts[user_counts >= item_min].index)

    filter_df = filter_df[user_subset].reset_index(drop=True)

    t_end = time()

#     assert (filter_df.groupby("user_id").size() < 5).sum() == 0
#     assert (filter_df.groupby("item_id").size() < 5).sum() == 0

    print(filter_df.nunique())
    print(filter_df.shape)
    print("{:.2f} Seconds".format(t_end - t_start))

    return filter_df

def reset_df(df):
    item_enc = LabelEncoder()
    df['news_id'] = df["item_id"]
    df["item_id"] = item_enc.fit_transform(df["item_id"])

    user_enc = LabelEncoder()
    df["user_id"] = user_enc.fit_transform(df["user_id"])

    assert df.user_id.min() == 0
    assert df.item_id.min() == 0

    return df

In [9]:
filtered_df = reset_df(filter_df(train_df, user_min, item_min))

user_id    20166
item_id    12550
dtype: int64
(4596882, 2)
0.96 Seconds


In [10]:
def convert_to_bpr_mat(dataframe, threshold = 3):
    tempdf = dataframe.copy()

    # Vì tập dữ liệu này đánh index từ 1 nên chuyển sang kiểu category
    # để tránh việc chúng ta có ma trận
    tempdf['user_id'] = tempdf['user_id'].astype('category')
    tempdf['item_id'] = tempdf['item_id'].astype('category')
    tempdf['positive'] = 1

    bpr_mat = csr_matrix((tempdf['positive'], (tempdf['user_id'].cat.codes, tempdf['item_id'].cat.codes)))
    bpr_mat.eliminate_zeros()
    del tempdf
    return bpr_mat

In [11]:
bpr_mat = convert_to_bpr_mat(filtered_df)

In [12]:
print(bpr_mat.shape)

(20166, 12550)


In [13]:
def split_to_train_test(bpr_mat, test_ratio = 0.2, verbose=True):
    """
    Chia tập dữ liệu ra thành tập train & tập test

    :param bpr_mat: ma trận bpr
    :param test_ratio: float - tỉ lệ test set

    :return train: ma trận bpr train
    :return test: ma trận bpr test
    """
    # Số lượng người dùng
    n_users = bpr_mat.shape[0]
    # Dùng ma trận thưa Dictionary Of Keys tối ưu hơn cho công đoạn này
    train = bpr_mat.copy().todok()
    test = dok_matrix(train.shape) # Lưu ý hiện tại test là ma trận 0

    # với mỗi người dùng u
    # chia số trường hợp nên khuyến nghị với tỉ lệ test_ratio đươc cho
    # phần nào thuộc về test
    for u in range(n_users):
        split_index = bpr_mat[u].indices
        # đếm số trường hợp nên khuyến nghị
        count_positive = split_index.shape[0]
        n_splits = max(min(math.ceil(test_ratio * count_positive), count_positive - 1), 1)
        test_index = np.random.choice(split_index, size=n_splits, replace=False)
        # Xem như dữ liệu chưa biết trong tập train
        train[u, test_index] = 0
        # Xem như dữ liệu nhìn thấy trong tập test
        test[u, test_index] = 1

    train, test = train.tocsr(), test.tocsr()

    # Nếu cần in thông tin ra ngoài
    if verbose:
        print('BPR matrix with %d stored elements' % bpr_mat.nnz)
        print('Train matrix with %d stored elements' % train.nnz)
        print('Test matrix with %d stored elements' % test.nnz)
    return train, test

In [14]:
bpr_train, bpr_test = split_to_train_test(bpr_mat, test_ratio=test_ratio, verbose=True)

BPR matrix with 666544 stored elements
Train matrix with 58723 stored elements
Test matrix with 607821 stored elements


In [15]:
def predict_bpr(W, H, user=None):
    """
    Hàm trả về X_hat

    :param W: ma trận W từ MF
    :param H: ma trận H từ MF
    :param user: người dùng (nếu None mặt định trả về tất cả)

    :return predict_scores: điểm dự đoán từ BPR MF
    """
    if user is None:
        return W @ H.T
    else:
        return W[user] @ H.T

def recommend_bpr(bpr_matrix, predict_score, user, n_rmd_items=None):
    """
    Dự đoán những sản phẩm mà người dùng muốn mua
    Những sản phẩm nào đã thích rồi thì không trả về nữa
    Trả về index theo bpr_matrix (đánh từ 0)

    :param bpr_matrix: ma trận bpr hiện tại
    :param predict_score: điểm dự đoán các item
    :param user: số thứ tự người dùng của predict score
    :param n_rmd_items: số lượng sản phẩm trả về, mặc định tất cả

    :return rmd_items: danh sách các sản phẩm khuyến nghị
    """
    # Số lượng sản phẩm
    n_items = bpr_matrix.shape[1]
    # những sản phẩm đã thích rồi
    liked_items = bpr_matrix[user].indices
    scores = predict_score.copy()

    # index ban đầu khi chưa sắp xếp
    sort_index = np.arange(0, n_items)

    # Xóa các sản phẩm đã mua
    sort_index = np.delete(sort_index, liked_items)
    scores = np.delete(scores, liked_items)

    # sắp xếp và trả về theo số thứ tự của score
    arg_sort = np.argsort(-scores)

    # dùng sort_index để lấy số thứ tự ban đầu
    rmd_items = sort_index[arg_sort]

    if len(rmd_items) >= n_rmd_items and n_rmd_items is not None:
        rmd_items = rmd_items[: n_rmd_items]
    return rmd_items

def auc_score(predict_mat, bpr_mat):
    """
    Tính Area under the ROC curve (AUC)
    cho bài toán hệ khuyến nghị

    :param predict_mat: ma trận dữ đoán bpr mf
    :param bpr_mat: ma trận train hoặc test
    :return auc: area under the roc curve
    """
    auc = 0.0
    n_users, n_items = bpr_mat.shape

    # u và row tương ứng user và bp
    for u in range(n_users):
        y_pred = predict_mat[u]
        y_true = np.zeros(n_items)
        y_true[bpr_mat[u].indices] = 1
        try:
            auc += roc_auc_score(y_true, y_pred)
        except ValueError:
            continue
    auc /= n_users
    return auc

In [16]:
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

def tanh(x):
    return 2 * sigmoid(2 * x) - 1

def learn_bpr_mf_sgd(bpr_mat, pos, neg, mapper, W = None, H = None, alpha=0.01, lamb=0.01, k=12, n_iters=10000, initial_value=2):
    """
    Thuật toán học BPR MF SGD (một điểm dữ liệu duy nhất)

    :param bpr_mat: ma trận bpr
    :param alpha: hệ số learning rate
    :param lamb: hệ số lambda của bình thường hóa regularization
    :param k: số lượng latent factor trong bài toán MF
    :param n_iters: số vòng lặp

    :return W: ma trận W
    :return H: ma trận H
    """
    
    # subset specifies
    print('initialize similarity model...', end=' ')
    sim_model = Similarity
    print('done!')
    
    n_users, n_items = bpr_mat.shape
    # Khởi tạo ma trận W và ma trận H
    if W is None:
        W = np.empty(shape=(n_users, k))
        W.fill(initial_value)
        
        W = np.random.rand(n_users, k)
    if H is None:
        H = np.empty(shape=(n_items, k))
        H.fill(initial_value)
        H = np.random.rand(n_items, k)

    # lặp
    for _ in range(n_iters):
        # ngẫu nghiên 3 bộ (u,i,j) từ D_S
        u = np.random.randint(0, n_users)
        if len(pos[u]) == 0:
            continue
        i = pos[u][np.random.randint(0, len(pos[u]))]
        j = neg[u][np.random.randint(0, len(neg[u]))]

        # Tính xuij
        xui = (W[u] * H[i]).sum()
        xuj = (W[u] * H[j]).sum()
        xuij = tanh(xui - xuj)
        
#         if (sim_model.similarity(mapper[i], mapper[j]) != 1):
#             xuij = xuij * (1 - sim_model.similarity(mapper[i], mapper[j]))

        # mũ tự nhiên e của xuij
        exp_xuij = np.exp(xuij)

        # sgd cho tham số Theta (W và H)
        W[u] = W[u] + alpha * ( exp_xuij / (1+exp_xuij) * (H[i] - H[j]) + lamb * W[u])
        H[i] = H[i] + alpha * ( exp_xuij / (1+exp_xuij) * W[u] + lamb * H[i])
        H[j] = H[j] + alpha * ( exp_xuij / (1+exp_xuij) * (-W[u]) + lamb * H[j])
    return W, H

In [17]:
W, H = None, None

# Tập các sản phẩm nên khuyến nghị
pos = np.split(bpr_train.indices, bpr_train.indptr)[1:-1]
# Tập các sản phẩm không nên khuyến nghị
neg = [np.setdiff1d(np.arange(0, bpr_train.shape[1], 1), e) for e in pos]

mapper = pd.Series(filtered_df["news_id"].values,index=filtered_df['item_id']).to_dict()

In [18]:
W, H = learn_bpr_mf_sgd(
    bpr_train,
    pos,
    neg,
    mapper,
    W = W,
    H = H,
    alpha=alpha,
    lamb=lamb,
    k=k,
    n_iters=n_iters,
    initial_value = initial_value
)

pred = predict_bpr(W, H)

train_score = auc_score(pred, bpr_train)
test_score = auc_score(pred, bpr_test)
print('AUC of train: %f' % train_score)
print('AUC of test : %f' % test_score)

initialize similarity model... done!
AUC of train: 0.669219
AUC of test : 0.652396


In [19]:
output_folder = os.path.join('./result', datetime.now().strftime("%d-%m-%Y %H-%M"))

pathlib.Path(output_folder).mkdir(parents=True, exist_ok=True)

np.save(os.path.join(output_folder, result_train_file + '_W.npy'), W)
np.save(os.path.join(output_folder, result_train_file + '_H.npy'), H)

with open(os.path.join(output_folder, result_train_file + '.txt'), 'w') as file:
    file.write("Parameters:\n\n")
    file.write('user min: %i' % user_min + "\n")
    file.write('item min: %i' % item_min + "\n")
    file.write('test ratio: %f' % test_ratio + "\n")
    file.write('alpha: %f' % alpha + "\n")
    file.write('lambda: %f' % lamb + "\n")
    file.write('k: %i' % k + "\n")
    file.write('n iters: %i' % n_iters + "\n")
    file.write('AUC-Train: %f' % train_score + "\n")
    file.write('AUC-Test: %f' % test_score + "\n")
    file.write("---------------------\n")
    file.write('Total users: %i \n' % train_df.nunique()[0])
    file.write('Total items: %i \n' % train_df.nunique()[1])
    file.write('Total interacts: %i \n' % train_df.shape[0])
    file.write('No training users: %i \n' % filtered_df.nunique()[0])
    file.write('No training items: %i \n' % filtered_df.nunique()[1])
    file.write('BPR matrix with %d stored elements\n' % bpr_mat.nnz)
    file.write('Train matrix with %d stored elements\n' % bpr_train.nnz)
    file.write('Test matrix with %d stored elements\n' % bpr_test.nnz)
    file.write('Matrix initial with %d' % initial_value)


In [20]:
u = 300
n_rmd_items = 5
score = predict_bpr(W, H, u)
rmd_items = recommend_bpr(bpr_train, score, u, n_rmd_items)
print(rmd_items)

[   46  6144  8256 10703   161]


In [21]:
result = (W @ H.T)

In [22]:
print(result.min())
print(result.max())
print(result.mean())

6.570187486663785
46.737755901691095
15.039560983331006


In [23]:
result

array([[15.27309718, 16.02198202, 14.57414907, ..., 13.09773195,
        12.09906051, 15.45219733],
       [15.10687976, 15.6346085 , 15.26079341, ..., 14.62642288,
        12.69462948, 15.4580041 ],
       [16.77814881, 14.95172002, 16.89846842, ..., 13.22637816,
        13.43220599, 15.47784065],
       ...,
       [15.64023993, 16.5201007 , 17.36820457, ..., 12.98550794,
        15.27807669, 15.63118303],
       [16.81492921, 17.36222565, 18.98478901, ..., 16.04367748,
        13.29970778, 17.29802435],
       [14.90673124, 16.41114687, 16.51085476, ..., 12.81657767,
        12.88300499, 15.8269745 ]])

# Tham khảo

01. Steffen Rendle, Christoph Freudenthaler, Zeno Gantner and Lars Schmidt-Thieme. BPR: Bayesian Personalized Ranking from Implicit Feedback. 
02. Weike Panand, Li Chen. GBPR: Group Preference Based Bayesian Personalized Ranking for One-Class Collaborative Filtering. Proceedings of the Twenty-Third International Joint Conference on Artificial Intelligence. https://www.aaai.org/ocs/index.php/IJCAI/IJCAI13/paper/viewFile/6316/7124
03. Michael D. Ekstrand, Joseph A Konstan. Personalized Ranking (with Daniel Kluver). Matrix Factorization and Advanced Techniques - University of Minnesota. https://www.coursera.org/lecture/matrix-factorization/personalized-ranking-with-daniel-kluver-s3XJo
04. Kim Falk. Practical Recommender Systems. Manning Publications.
05. Ethen (MingYu) Liu. Bayesian Personalized Ranking. http://ethen8181.github.io/machine-learning/recsys/4_bpr.html
06. Alfredo Láinez Rodrigo, Luke de Oliveira. Distributed Bayesian Personalized Ranking in Spark. https://stanford.edu/~rezab/classes/cme323/S16/projects_reports/rodrigo_oliveira.pdf